In [1]:
import json
import time 
import pandas as pd
import os
from kafka import KafkaProducer
from pyspark.sql import functions as F

### Question 1

In [2]:
!docker exec redpanda-1 rpk version

v22.3.5 (rev 28b2443)


### Question 2

In [3]:
!docker exec redpanda-1 rpk topic create test-topic

TOPIC       STATUS
test-topic  OK


### Question 3

In [4]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [5]:
server = 'localhost:9092'

In [6]:
producer = KafkaProducer(
    bootstrap_servers = [server],
    value_serializer = json_serializer
    )

In [7]:
producer.bootstrap_connected()

True

### Question 4

In [8]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
t2 = time.time()
producer.flush()

t1 = time.time()
print(f'Streaming took {(t2 - t0):.2f} seconds')
print(f'Flushing took {(t1 - t2):.2f} seconds')
print(f'Total time took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
Streaming took 0.52 seconds
Flushing took 0.00 seconds
Total time took 0.52 seconds


### Question 5

In [9]:
if not os.path.isfile('green_tripdata_2019-10.csv.gz'):
    !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

In [10]:
columns_needed = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', \
                  'trip_distance', 'tip_amount']
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', usecols=columns_needed)

In [11]:
df_green.head(5)

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [12]:
!docker exec redpanda-1 rpk topic create green-trips

TOPIC        STATUS
green-trips  OK


In [14]:
t0 = time.time()
topic_name = 'green-trips'
for row in df_green.itertuples(index=False):
    message = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=message)
producer.flush()

t1 = time.time()

print(f'Total time took {(t1 - t0):.2f} seconds')

Total time took 175.82 seconds
